# FIT5120 - Industry Experience Studio Project

## Backpackers' Mate - Team A17 - TASSWARE

### Iteration 2 - Data Wrangling/Integration/ERD tables

### Introduction

In this iteration, the team came up with a way to use the climate data and representing it to the user in the form of visualization in a way they can understand. The whole idea is to let the user, in our case, the backpackers, aware about each regional climate conditions. Victoria has a capricious climate and things can take a turn at any point of time. The working conditions experienced by outdoor workers are harsh most of the time. And the conditions are not the same for all regions. Each of the regions in different parts of Victoria has a different behavioural pattern. Our visualizations will represent the climate trend for different region. In order to create the visualization, we have gathered data from Australian Bureau of Meteorology. BOM has open data and has a copyright creative commons 3.0. It has been allowed to use by the public. The data has been extracted from 15 different farming regions and contains different attributes.

### Libraries

In [33]:
import pandas as pd
import datetime

### Using old tables

We shall use the tables which have already been created and use it for merging and concatenating operations.

In [34]:
# Reading data
farming_region = pd.read_excel('FarmingRegion.xlsx', encoding = 'utf-8', sep = ',')

# Renaming column
farming_region = farming_region.rename(columns = {'Farming Regions Name': 'FarmingRegion'})
farming_region.head()

ID FarmingRegion                                        Description
0   1    Bairnsdale  Bairnsdale is located on the river flats of th...
1   2    Beechworth  Beechworth is one of Victoria’s better preserv...
2   3        Cobram  Cobram is located on the Murray River, which i...
3   4       Mildura  Mildura is a regional city and a popular touri...
4   5        Echuca  Often referred to as Echuca Moama, Echuca is l...

In [35]:
regions = farming_region['FarmingRegion'].tolist()

### Data Wrangling

Let us read one of the CSV files and wrangle them. We shall get it into a format where it has the station number with yearly maximum and minimum temperatures number of days.

In [36]:
# Read the CSV
data = pd.read_csv('Mildura.csv')
data.head()

Product code  Bureau of Meteorology station number  Year  Month  Day  \
0   IDCJAC0010                                 76031  1946      1    1   
1   IDCJAC0010                                 76031  1946      1    2   
2   IDCJAC0010                                 76031  1946      1    3   
3   IDCJAC0010                                 76031  1946      1    4   
4   IDCJAC0010                                 76031  1946      1    5   

   Maximum temperature (Degree C)  \
0                             NaN   
1                             NaN   
2                             NaN   
3                             NaN   
4                             NaN   

   Days of accumulation of maximum temperature Quality  
0                                          NaN     NaN  
1                                          NaN     NaN  
2                                          NaN     NaN  
3                                          NaN     NaN  
4                                          NaN     NaN

In [37]:
# Drop Null 
data = data.dropna(axis=0, subset=['Maximum temperature (Degree C)'])

# Remove data points for value = N (N is not quality assured data)
data = data[~data.Quality.str.contains("N")]

# Dropping unnecessary columns
data.drop(columns = ['Days of accumulation of maximum temperature', 'Quality'], inplace = True)
data.head()

Product code  Bureau of Meteorology station number  Year  Month  Day  \
238   IDCJAC0010                                 76031  1946      8   27   
239   IDCJAC0010                                 76031  1946      8   28   
240   IDCJAC0010                                 76031  1946      8   29   
241   IDCJAC0010                                 76031  1946      8   30   
242   IDCJAC0010                                 76031  1946      8   31   

     Maximum temperature (Degree C)  
238                            14.8  
239                            14.5  
240                            16.3  
241                            18.3  
242                            17.0

In [38]:
# Filtering the data out inro summer and winter months
zzz = data[(data['Month'] >= 11) | (data['Month'] <=3 )]

# Filtering out for temperature more than 30 degrees
zzz = zzz[(data['Maximum temperature (Degree C)']>30)]

# Grouping by Year
zzz = zzz.groupby(zzz.Year)['Maximum temperature (Degree C)'].agg('count').reset_index()

# Small changes to the resulting table
zzz = zzz.rename(columns= {'Maximum temperature (Degree C)' : 'DaysOver30'})
zzz['FarmingRegion'] = 'Mildura'
zzz = zzz[['FarmingRegion','Year','DaysOver30']]
zzz.head()

C:\Users\anman\Anaconda3\envs\DAA\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


FarmingRegion  Year  DaysOver30
0       Mildura  1946          24
1       Mildura  1947          67
2       Mildura  1948          64
3       Mildura  1949          44
4       Mildura  1950          65

In [39]:
# Same operations but this is for winter months lesser than 18 degrees
xxx = data[(data['Month'] >= 3) | (data['Month'] <=11 )]
xxx = xxx[(xxx['Maximum temperature (Degree C)']<18)]
xxx = xxx.groupby(xxx.Year)['Maximum temperature (Degree C)'].agg('count').reset_index()
xxx = xxx.rename(columns= {'Maximum temperature (Degree C)' : 'DaysUnder18'})
xxx = xxx[['Year','DaysUnder18']]
xxx.head()

Year  DaysUnder18
0  1946           17
1  1947           84
2  1948          108
3  1949          112
4  1950           85

In [40]:
# Merging both the tables
df = pd.merge(zzz, xxx, on='Year')
df = df.dropna()
df.head()

FarmingRegion  Year  DaysOver30  DaysUnder18
0       Mildura  1946          24           17
1       Mildura  1947          67           84
2       Mildura  1948          64          108
3       Mildura  1949          44          112
4       Mildura  1950          65           85

We need the above format for all the given regions stacked on top of each other in a table.

###  Merging all the data

Let us do that by writing a single block of code which will take all the 15 regions climate data and combines it to form a single table.

In [41]:
# Creating an empty dataframe
d = {}
pola = pd.DataFrame(data=d)

# Looping through all the regions
for i in regions:
    
    # Performing the same operations as above
    data = pd.read_csv(str(i)+'.csv')
    data = data.dropna(axis=0, subset=['Maximum temperature (Degree C)'])
    data = data[~data.Quality.str.contains("N")]
    data.drop(columns = ['Days of accumulation of maximum temperature', 'Quality'], inplace = True)
    
    zzz = data[(data['Month'] >= 11) | (data['Month'] <=3 )]
    zzz = zzz[(data['Maximum temperature (Degree C)']>30)]
    zzz = zzz.groupby(zzz.Year)['Maximum temperature (Degree C)'].agg('count').reset_index()
    zzz = zzz.rename(columns= {'Maximum temperature (Degree C)' : 'DaysOver30'})
    zzz['FarmingRegion'] = str(i)
    zzz = zzz[['FarmingRegion','Year','DaysOver30']]
        
    xxx = data[(data['Month'] >= 3) | (data['Month'] <=11 )]
    xxx = xxx[(xxx['Maximum temperature (Degree C)']<18)]
    xxx = xxx.groupby(xxx.Year)['Maximum temperature (Degree C)'].agg('count').reset_index()
    xxx = xxx.rename(columns= {'Maximum temperature (Degree C)' : 'DaysUnder18'})
    xxx = xxx[['Year','DaysUnder18']]

    df = pd.merge(zzz, xxx, on='Year')
    df = df.dropna()
    
    # Appending each wrangled region data to the empty dataframe
    pola = pola.append(df)

# Reset index
pola = pola.reset_index(drop=True)
pola    

C:\Users\anman\Anaconda3\envs\DAA\lib\site-packages\ipykernel_launcher.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


FarmingRegion  Year  DaysOver30  DaysUnder18
0      Bairnsdale  1943           4          172
1      Bairnsdale  1944          21          161
2      Bairnsdale  1945          11          167
3      Bairnsdale  1983           7           56
4      Bairnsdale  1984          13          151
..            ...   ...         ...          ...
795    Rutherglen  2015          54          132
796    Rutherglen  2016          82           88
797    Rutherglen  2017          67           43
798    Rutherglen  2018          59          114
799    Rutherglen  2019          35           74

[800 rows x 4 columns]

In [42]:
# Small changes and merging with the old table to change the format
data = pd.merge(pola, farming_region, how = 'left', on = 'FarmingRegion')
data.drop(columns = ['Description'], inplace = True)
data = data[['ID','FarmingRegion','Year','DaysOver30','DaysUnder18']]
data.head()

ID FarmingRegion  Year  DaysOver30  DaysUnder18
0   1    Bairnsdale  1943           4          172
1   1    Bairnsdale  1944          21          161
2   1    Bairnsdale  1945          11          167
3   1    Bairnsdale  1983           7           56
4   1    Bairnsdale  1984          13          151

Let us now filter out the past 10 year data from all of the regions in the above table.

In [43]:
month = ['Jan','Feb','Mar','Apr','May','Jun','Jul', 'Aug','Sep','Oct','Nov','Dec']
all_year = data.Year.unique().tolist()
viz_year = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
remove_year = [x for x in all_year if x not in viz_year]

# Removing years which are not past 10 years
for year in remove_year:
    data = data[data.Year != year]
    
data.head()


ID FarmingRegion  Year  DaysOver30  DaysUnder18
30   1    Bairnsdale  2010          28          157
31   1    Bairnsdale  2011          15          135
32   1    Bairnsdale  2012          28          145
33   1    Bairnsdale  2013          33          124
34   1    Bairnsdale  2014          26          127

In [44]:
# Saving the data a new ERD table 
data.to_csv('ClimateHistory.csv')

### Another table for Monthly averages

Let us create another table which has an average monthly maximum temperature for the past 10 years for each of the regions.

In [45]:
# Reading the data
data = pd.read_csv('Mildura.csv')
data.head()

Product code  Bureau of Meteorology station number  Year  Month  Day  \
0   IDCJAC0010                                 76031  1946      1    1   
1   IDCJAC0010                                 76031  1946      1    2   
2   IDCJAC0010                                 76031  1946      1    3   
3   IDCJAC0010                                 76031  1946      1    4   
4   IDCJAC0010                                 76031  1946      1    5   

   Maximum temperature (Degree C)  \
0                             NaN   
1                             NaN   
2                             NaN   
3                             NaN   
4                             NaN   

   Days of accumulation of maximum temperature Quality  
0                                          NaN     NaN  
1                                          NaN     NaN  
2                                          NaN     NaN  
3                                          NaN     NaN  
4                                          NaN     NaN

In [46]:
# Same Initial Wrangling operations
data = data.dropna(axis=0, subset=['Maximum temperature (Degree C)'])
data = data[~data.Quality.str.contains("N")]
data.drop(columns = ['Days of accumulation of maximum temperature', 'Quality'], inplace = True)

# Removing unnecessary years
for year in remove_year:
    data = data[data.Year != year]

data.head()

Product code  Bureau of Meteorology station number  Year  Month  Day  \
23376   IDCJAC0010                                 76031  2010      1    1   
23377   IDCJAC0010                                 76031  2010      1    2   
23378   IDCJAC0010                                 76031  2010      1    3   
23379   IDCJAC0010                                 76031  2010      1    4   
23380   IDCJAC0010                                 76031  2010      1    5   

       Maximum temperature (Degree C)  
23376                            37.4  
23377                            27.4  
23378                            27.2  
23379                            30.9  
23380                            36.3

In [47]:
# Aggregating it by average
zzz = data.copy()
zzz.drop(columns = ['Product code', 'Bureau of Meteorology station number'])
zzz = zzz.groupby(zzz.Month)['Maximum temperature (Degree C)'].agg('mean').reset_index()
zzz = zzz.rename(columns= {'Maximum temperature (Degree C)' : 'Avg.MaxTemp', 'Month': 'MonthId'})
zzz['FarmingRegion'] = 'Mildura'
zzz['Month'] = month
zzz = zzz[['FarmingRegion','MonthId','Month','Avg.MaxTemp']]

### Monthly Averages all region data

Let's write a similar block of code that brings in all the montly averages of maximum temperature and has all the 15 farming region data.

In [48]:
d = {}
pola = pd.DataFrame(data=d)

for i in regions:
    
    data = pd.read_csv(str(i)+'.csv')
    data = data.dropna(axis=0, subset=['Maximum temperature (Degree C)'])
    data = data[~data.Quality.str.contains("N")]
    data.drop(columns = ['Days of accumulation of maximum temperature', 'Quality'], inplace = True)
    
    zzz = data.copy()
    zzz.drop(columns = ['Product code', 'Bureau of Meteorology station number'])
    zzz = zzz.groupby(zzz.Month)['Maximum temperature (Degree C)'].agg('mean').reset_index()
    zzz = zzz.rename(columns= {'Maximum temperature (Degree C)' : 'Avg.MaxTemp', 'Month': 'MonthId'})
    zzz['FarmingRegion'] = str(i)
    zzz['Month'] = month
    zzz = zzz[['FarmingRegion','MonthId','Month','Avg.MaxTemp']]

    zzz = zzz.dropna()
    
    pola = pola.append(zzz)

pola = pola.reset_index(drop=True)
pola    

FarmingRegion  MonthId Month  Avg.MaxTemp
0      Bairnsdale        1   Jan    25.961453
1      Bairnsdale        2   Feb    25.584971
2      Bairnsdale        3   Mar    23.946010
3      Bairnsdale        4   Apr    20.704676
4      Bairnsdale        5   May    17.644482
..            ...      ...   ...          ...
175    Rutherglen        8   Aug    14.055803
176    Rutherglen        9   Sep    17.217047
177    Rutherglen       10   Oct    20.997323
178    Rutherglen       11   Nov    25.368848
179    Rutherglen       12   Dec    29.240061

[180 rows x 4 columns]

In [49]:
# Bringing it to the correct format
df = pd.merge(pola, farming_region, how = 'left', on = 'FarmingRegion')
df.drop(columns = ['Description'], inplace = True)
df = df.rename(columns = {'ID':'FarmingRegionId'})
df = df[['FarmingRegionId', 'FarmingRegion', 'MonthId','Month','Avg.MaxTemp']]
df.head()

FarmingRegionId FarmingRegion  MonthId Month  Avg.MaxTemp
0                1    Bairnsdale        1   Jan    25.961453
1                1    Bairnsdale        2   Feb    25.584971
2                1    Bairnsdale        3   Mar    23.946010
3                1    Bairnsdale        4   Apr    20.704676
4                1    Bairnsdale        5   May    17.644482

In [50]:
# Saving it as another table for ERD as Monthly averages
df.to_csv('Monthlyavgs.csv')

### Data Wrangling of Nearest Medical center

The following data was obtained from data.gov.au which had hospital locations across Victoria. We shall use this data in order to represent it in Google Maps API. 

In [51]:
# Reading the Data
med_data = pd.read_csv('Hospital_Locations_1.csv')
print("Shape: ", med_data.shape) 
med_data.head()

Shape:  (325, 17)


X          Y  FID                                    LabelName  \
0  144.983399 -37.809681    1       East Melbourne Specialist Day Hospital   
1  145.149300 -37.951400    2                   Windsor Avenue Day Surgery   
2  145.344653 -38.034580    3       Hyperbaric Health Wound Centre Berwick   
3  143.832286 -37.540553    4                Ballarat Day Procedure Centre   
4  145.143379 -38.148189    5  Bayside Day Procedure and Specialist Centre   

                         OpsName     Type  StreetNum    RoadName RoadType  \
0    EAST MELBOURNE DAY HOSPITAL  PRIVATE         23   CLARENDON   STREET   
1     WINDSOR AVENUE DAY SURGERY  PRIVATE         17     WINDSOR   AVENUE   
2    HYPERBARIC HEALTH - BERWICK  PRIVATE          3        GIBB   STREET   
3  BALLARAT DAY PROCEDURE CENTRE  PRIVATE  1117-1123      HOWITT   STREET   
4   BAYSIDE DAY PROCEDURE CENTRE  PRIVATE        141  CRANBOURNE     ROAD   

  RoadSuffix  CampusCode            LGAName LocalityNa  Postcode VicgovRegi  \
0                   8280          MELBOURNE                 3002       NWMR   
1                   7050  GREATER DANDENONG                 3171        SMR   
2                   7100              CASEY                 3806        SMR   
3                   8430           BALLARAT                 3355       Gram   
4                   7310          FRANKSTON                 3199        SMR   

  State ServiceNam  
0   VIC             
1   VIC             
2   VIC             
3   VIC             
4   VIC

In [52]:
# Dropping Unnecessary columns
med_data.drop(columns = ['FID','OpsName','RoadSuffix','CampusCode','VicgovRegi','ServiceNam'], inplace = True)

# Joining address columns to create a single column of the whole Address
med_data['StreetName'] = med_data[['RoadName', 'RoadType']].agg(' '.join, axis=1)
med_data['Street'] = med_data[['StreetNum', 'StreetName']].agg(' '.join, axis=1)
med_data['Address_1'] = med_data[['Street', 'LGAName']].agg(', '.join, axis=1)
med_data['Address_2'] = med_data[['Address_1', 'State']].agg(', '.join, axis=1)
med_data['code'] = med_data['Postcode'].astype(str)
med_data['Address_3'] = med_data[['Address_2', 'code']].agg(' '.join, axis=1)
med_data.head()


X          Y                                    LabelName  \
0  144.983399 -37.809681       East Melbourne Specialist Day Hospital   
1  145.149300 -37.951400                   Windsor Avenue Day Surgery   
2  145.344653 -38.034580       Hyperbaric Health Wound Centre Berwick   
3  143.832286 -37.540553                Ballarat Day Procedure Centre   
4  145.143379 -38.148189  Bayside Day Procedure and Specialist Centre   

      Type  StreetNum    RoadName RoadType            LGAName LocalityNa  \
0  PRIVATE         23   CLARENDON   STREET          MELBOURNE              
1  PRIVATE         17     WINDSOR   AVENUE  GREATER DANDENONG              
2  PRIVATE          3        GIBB   STREET              CASEY              
3  PRIVATE  1117-1123      HOWITT   STREET           BALLARAT              
4  PRIVATE        141  CRANBOURNE     ROAD          FRANKSTON              

   Postcode State        StreetName                   Street  \
0      3002   VIC  CLARENDON STREET      23 CLARENDON STREET   
1      3171   VIC    WINDSOR AVENUE        17 WINDSOR AVENUE   
2      3806   VIC       GIBB STREET            3 GIBB STREET   
3      3355   VIC     HOWITT STREET  1117-1123 HOWITT STREET   
4      3199   VIC   CRANBOURNE ROAD      141 CRANBOURNE ROAD   

                              Address_1  \
0        23 CLARENDON STREET, MELBOURNE   
1  17 WINDSOR AVENUE, GREATER DANDENONG   
2                  3 GIBB STREET, CASEY   
3     1117-1123 HOWITT STREET, BALLARAT   
4        141 CRANBOURNE ROAD, FRANKSTON   

                                   Address_2  code  \
0        23 CLARENDON STREET, MELBOURNE, VIC  3002   
1  17 WINDSOR AVENUE, GREATER DANDENONG, VIC  3171   
2                  3 GIBB STREET, CASEY, VIC  3806   
3     1117-1123 HOWITT STREET, BALLARAT, VIC  3355   
4        141 CRANBOURNE ROAD, FRANKSTON, VIC  3199   

                                        Address_3  
0        23 CLARENDON STREET, MELBOURNE, VIC 3002  
1  17 WINDSOR AVENUE, GREATER DANDENONG, VIC 3171  
2                  3 GIBB STREET, CASEY, VIC 3806  
3     1117-1123 HOWITT STREET, BALLARAT, VIC 3355  
4        141 CRANBOURNE ROAD, FRANKSTON, VIC 3199

In [53]:
# Drop unnecessary columns
med_data.drop(columns = ['StreetNum','RoadName','RoadType','LGAName','LocalityNa','State','StreetName','Street','Address_1','Address_2','code'], inplace = True)

# Rename columns 
med_data = med_data.rename(columns = {'Address_3':'Address'})
med_data['Hospital'] = 'HOSPITAL'

# Reformating the data
med_data['T'] = med_data[['Type', 'Hospital']].agg(' '.join, axis=1)
med_data.drop(columns = ['Type', 'Hospital'], inplace = True)
med_data = med_data.rename(columns = {'T':'Type', 'LabelName':'Name', 'X':'Longitude', 'Y':'Latitude'})
med_data.head()

Longitude   Latitude                                         Name  \
0  144.983399 -37.809681       East Melbourne Specialist Day Hospital   
1  145.149300 -37.951400                   Windsor Avenue Day Surgery   
2  145.344653 -38.034580       Hyperbaric Health Wound Centre Berwick   
3  143.832286 -37.540553                Ballarat Day Procedure Centre   
4  145.143379 -38.148189  Bayside Day Procedure and Specialist Centre   

   Postcode                                         Address              Type  
0      3002        23 CLARENDON STREET, MELBOURNE, VIC 3002  PRIVATE HOSPITAL  
1      3171  17 WINDSOR AVENUE, GREATER DANDENONG, VIC 3171  PRIVATE HOSPITAL  
2      3806                  3 GIBB STREET, CASEY, VIC 3806  PRIVATE HOSPITAL  
3      3355     1117-1123 HOWITT STREET, BALLARAT, VIC 3355  PRIVATE HOSPITAL  
4      3199        141 CRANBOURNE ROAD, FRANKSTON, VIC 3199  PRIVATE HOSPITAL

In [54]:
# Removing hospitals where there is a mention if Surgery, Endoscopy, and others which won't be of any use to the user.
med_data = med_data[~med_data.Name.str.contains("Surgery")]
med_data = med_data[~med_data.Name.str.contains("Surgicentre")]
med_data = med_data[~med_data.Name.str.contains("Endoscopy")]
med_data = med_data[~med_data.Name.str.contains("Dialysis")]
med_data = med_data[~med_data.Name.str.contains("Rehabiliation")]
med_data = med_data[~med_data.Name.str.contains("Gastroenterology")]
med_data = med_data[~med_data.Name.str.contains("Imaging")]
med_data = med_data[~med_data.Name.str.contains("Radiation")]
med_data = med_data[~med_data.Name.str.contains("Oncology")]
med_data = med_data[~med_data.Name.str.contains("Oral")]
med_data = med_data[~med_data.Name.str.contains("Laser")]
med_data = med_data[~med_data.Name.str.contains("Facial")]
med_data = med_data[~med_data.Name.str.contains("Cosmetic")]
med_data = med_data[~med_data.Name.str.contains("Beauty")]
med_data = med_data[~med_data.Name.str.contains("Eye")]
med_data = med_data[~med_data.Name.str.contains("Ear")]
med_data

Longitude   Latitude                                               Name  \
0    144.983399 -37.809681             East Melbourne Specialist Day Hospital   
2    145.344653 -38.034580             Hyperbaric Health Wound Centre Berwick   
3    143.832286 -37.540553                      Ballarat Day Procedure Centre   
4    145.143379 -38.148189        Bayside Day Procedure and Specialist Centre   
9    145.061879 -37.692573                               Victorian Gut Centre   
..          ...        ...                                                ...   
319  146.939700 -36.078500                     Albury Wodonga Health (Albury)   
320  142.857700 -35.500500  Mallee Track Health & Community Service (Sea L...   
321  144.887100 -37.791100                              Orygen Inpatient Unit   
322  145.018500 -37.961000                                Women's Sandringham   
323  144.886976 -37.791622                               Ursula Frayne Centre   

     Postcode                                        Address              Type  
0        3002       23 CLARENDON STREET, MELBOURNE, VIC 3002  PRIVATE HOSPITAL  
2        3806                 3 GIBB STREET, CASEY, VIC 3806  PRIVATE HOSPITAL  
3        3355    1117-1123 HOWITT STREET, BALLARAT, VIC 3355  PRIVATE HOSPITAL  
4        3199       141 CRANBOURNE ROAD, FRANKSTON, VIC 3199  PRIVATE HOSPITAL  
9        3083          119 PLENTY ROAD, WHITTLESEA, VIC 3083  PRIVATE HOSPITAL  
..        ...                                            ...               ...  
319      2640  201 BORELLA ROAD, EAST ALBURY (NSW), NSW 2640   PUBLIC HOSPITAL  
320      3533    33-43 MCCLELLAND AVENUE, SEA LAKE, VIC 3533   PUBLIC HOSPITAL  
321      3011           10 MAVIS STREET, FOOTSCRAY, VIC 3011   PUBLIC HOSPITAL  
322      3191      191-193 BLUFF ROAD, SANDRINGHAM, VIC 3191   PUBLIC HOSPITAL  
323      3011         160 GORDON STREET, FOOTSCRAY, VIC 3011   PUBLIC HOSPITAL  

[261 rows x 6 columns]

In [55]:
# Re-formatting the data 
med_data = med_data[['Postcode', 'Name', 'Latitude','Longitude', 'Address',"Type"]]
med_data = med_data.reset_index(drop = True)
med_data.head()

Postcode                                         Name   Latitude  \
0      3002       East Melbourne Specialist Day Hospital -37.809681   
1      3806       Hyperbaric Health Wound Centre Berwick -38.034580   
2      3355                Ballarat Day Procedure Centre -37.540553   
3      3199  Bayside Day Procedure and Specialist Centre -38.148189   
4      3083                         Victorian Gut Centre -37.692573   

    Longitude                                      Address              Type  
0  144.983399     23 CLARENDON STREET, MELBOURNE, VIC 3002  PRIVATE HOSPITAL  
1  145.344653               3 GIBB STREET, CASEY, VIC 3806  PRIVATE HOSPITAL  
2  143.832286  1117-1123 HOWITT STREET, BALLARAT, VIC 3355  PRIVATE HOSPITAL  
3  145.143379     141 CRANBOURNE ROAD, FRANKSTON, VIC 3199  PRIVATE HOSPITAL  
4  145.061879        119 PLENTY ROAD, WHITTLESEA, VIC 3083  PRIVATE HOSPITAL

In [56]:
# Saving the above table to be in the ERD
med_data.to_csv('MedicalServices.csv')

### Wrangling of Heat Stress Dataset 

We shall wrangle another dataset which has important climate parameters which we can indicate the heat stress levels. We have to get it in the correct format so that we can run a machine learning algorithm on it.

In [57]:
# Read data and check the column types
data = pd.read_csv("Hourly_Data_-_44_Percy_Street_Warwick_-_WBGT.csv", skiprows=2,encoding='utf-8' )
data = data.rename(columns = {';Date': 'Date'})
data.dtypes

Date                                       object
 Time                                      object
 MAXIMUM Wind Speed 10m km/h              float64
AVERAGE Wind Speed 10m km/h               float64
STDEV Wind Speed 10m km/h                 float64
AVERAGE Wind Direction 10m Degrees        float64
AVERAGE Wind Direction 10m DESCRIPTION     object
S THETA Wind Direction 10m Degrees        float64
MAXIMUM Peak Wind Gust km/h               float64
AVERAGE Relative Humidity %               float64
AVERAGE Air Temperature DegC              float64
TOTAL Rain Gauge mm                       float64
AVERAGE Solar Radiation W/m2              float64
AVERAGE Barometric Pressure hPa           float64
AVERAGE Grass Temperature DegC            float64
MAXIMUM Wind Speed 2m km/h                float64
AVERAGE Wind Speed 2m km/h                float64
AVERAGE Black Globe Temp DegC             float64
AVERAGE Baro Pressure kPa kPa             float64
AVERAGE Wet Bulb Temp DegC                float64


In [58]:
# Grabbing only datapoints
data = data.iloc[1:38307]
col = data.columns
a = list(set(col) - set(['Date']))

# Setting the date format correct and extracting hour of the day
data['Date'] = pd.to_datetime(data['Date'], format='%Y/%m/%d')
data['hour_of_the_day'] = pd.to_datetime(data[' Time'], format='%H:%M:%S').dt.hour
data = data[['Date', 'hour_of_the_day']+a]
data.head()


Date  hour_of_the_day  AVERAGE Wind Direction 10m Degrees  \
1 2015-08-13               14                              247.85   
2 2015-08-13               15                              245.44   
3 2015-08-13               16                              251.03   
4 2015-08-13               17                              257.41   
5 2015-08-13               18                              253.15   

   AVERAGE Relative Humidity %  AVERAGE Wind Speed 10m km/h  \
1                        37.03                        21.72   
2                        37.90                        21.61   
3                        37.90                        16.47   
4                        40.24                        13.67   
5                        45.85                        10.42   

   AVERAGE Battery Voltage V      Time  AVGDIR Wind Direction 10m Degrees  \
1                      13.98  14:00:00                                NaN   
2                      13.98  15:00:00                                NaN   
3                      13.98  16:00:00                                NaN   
4                      13.85  17:00:00                                NaN   
5                      13.08  18:00:00                                NaN   

    MAXIMUM Wind Speed 10m km/h  AVERAGE Solar Voltage V  ...  \
1                         31.24                    20.95  ...   
2                         33.01                    20.93  ...   
3                         26.09                    20.67  ...   
4                         21.00                    18.73  ...   
5                         18.46                     9.13  ...   

   MAXIMUM Peak Wind Gust km/h  AVERAGE Wet Bulb Temp DegC  \
1                        47.33                        8.67   
2                        41.41                        8.85   
3                        34.31                        8.82   
4                        28.40                        8.52   
5                        23.66                        7.69   

   CURRENT Accum Heat Load Unit 83 HLU  AVGDIR Wind Direction 10m DESCRIPTION  \
1                                  NaN                                      N   
2                                  NaN                                      N   
3                                  NaN                                      N   
4                                  NaN                                      N   
5                                  NaN                                      N   

   STDEV Wind Speed 10m km/h  TOTAL Rain Gauge mm  \
1                       4.16                  0.0   
2                       4.21                  0.0   
3                       4.00                  0.0   
4                       2.77                  0.0   
5                       3.25                  0.0   

   CURRENT Evaporation Rate mm/h  MAXIMUM Wind Speed 2m km/h  \
1                         0.3116                       18.76   
2                         0.2712                       18.64   
3                         0.2027                       16.52   
4                         0.1218                       12.21   
5                         0.0621                       10.38   

  AVERAGE Air Temperature DegC  AVERAGE Thermal Work Limit W/m2  
1                        16.01                           432.76  
2                        16.08                           434.46  
3                        16.05                           428.28  
4                        15.25                           448.64  
5                        13.34                           483.71  

[5 rows x 35 columns]

In [59]:
# Dropping unnecessary columns
data.drop(columns = ['Date',' MAXIMUM Wind Speed 10m km/h', 'CURRENT Heat Load Index HLU', 'AVERAGE Grass Temperature DegC', 'MAXIMUM Wind Speed 2m km/h', 'AVERAGE Battery Voltage V', 'CURRENT Accum Heat Load Unit 90 HLU', 'STDEV Wind Speed 10m km/h', 'CURRENT Accum Heat Load Unit 86 HLU', 'AVGDIR Wind Direction 10m DESCRIPTION', ' Time', 'AVERAGE Wind Direction 10m DESCRIPTION', 'CURRENT Accum Heat Load Unit 92 HLU', 'MAXIMUM Peak Wind Gust km/h', 'CURRENT Accum Heat Load Unit 83 HLU', 'S THETA Wind Direction 10m Degrees', 'CURRENT Accum Heat Load Unit HLU', 'AVGDIR Wind Direction 10m Degrees', 'CURRENT AHLU 86 HLU', 'AVERAGE Wind Speed 10m km/h'], inplace = True)
data.head()

hour_of_the_day  AVERAGE Wind Direction 10m Degrees  \
1               14                              247.85   
2               15                              245.44   
3               16                              251.03   
4               17                              257.41   
5               18                              253.15   

   AVERAGE Relative Humidity %  AVERAGE Solar Voltage V  \
1                        37.03                    20.95   
2                        37.90                    20.93   
3                        37.90                    20.67   
4                        40.24                    18.73   
5                        45.85                     9.13   

   AVERAGE Barometric Pressure hPa  AVERAGE Wet Bulb Globe Temp DegC  \
1                          1008.53                             12.02   
2                          1008.71                             12.11   
3                          1009.15                             12.07   
4                          1009.97                             11.15   
5                          1011.00                              9.15   

   AVERAGE Baro Pressure kPa kPa  AVERAGE Black Globe Temp DegC  \
1                          95.45                          21.73   
2                          95.47                          21.52   
3                          95.52                          21.42   
4                          95.60                          18.16   
5                          95.70                          12.08   

   AVERAGE Solar Radiation W/m2  AVERAGE Wind Speed 2m km/h  \
1                        454.02                       13.05   
2                        361.33                       12.90   
3                        239.80                        9.60   
4                        102.72                        7.36   
5                          7.63                        5.44   

   AVERAGE Wet Bulb Temp DegC  TOTAL Rain Gauge mm  \
1                        8.67                  0.0   
2                        8.85                  0.0   
3                        8.82                  0.0   
4                        8.52                  0.0   
5                        7.69                  0.0   

   CURRENT Evaporation Rate mm/h  AVERAGE Air Temperature DegC  \
1                         0.3116                         16.01   
2                         0.2712                         16.08   
3                         0.2027                         16.05   
4                         0.1218                         15.25   
5                         0.0621                         13.34   

   AVERAGE Thermal Work Limit W/m2  
1                           432.76  
2                           434.46  
3                           428.28  
4                           448.64  
5                           483.71

In [60]:
# Saving the above table as a sample data
data.to_csv('sample_test_heat_stress_v1.csv')

The above table has the parameters used to build the machine learning algorithm and later, we shall choose the variables required for machine learning model. 

Based on the Machine Learning model (as displayed in the other file), we get to know that four parameters are enough for prediction of the Heat Stress level,i.e., WBGT. Hence, we shall remove the rest of the variables from the table and save it as the table to be uploaded to the database in ERD where we will build a machine learning model for prediction of Heat Stress based on Live weather feed in Visual Studio ASP.NET application C# platform.

In [61]:
data.drop(columns = ['hour_of_the_day', 'AVERAGE Thermal Work Limit W/m2', 'AVERAGE Solar Radiation W/m2', 'AVERAGE Black Globe Temp DegC', 'AVERAGE Baro Pressure kPa kPa', 'CURRENT Evaporation Rate mm/h', 'AVERAGE Solar Voltage V', 'AVERAGE Wind Direction 10m Degrees', 'AVERAGE Wet Bulb Temp DegC', 'TOTAL Rain Gauge mm'], inplace = True)
data.head()

AVERAGE Relative Humidity %  AVERAGE Barometric Pressure hPa  \
1                        37.03                          1008.53   
2                        37.90                          1008.71   
3                        37.90                          1009.15   
4                        40.24                          1009.97   
5                        45.85                          1011.00   

   AVERAGE Wet Bulb Globe Temp DegC  AVERAGE Wind Speed 2m km/h  \
1                             12.02                       13.05   
2                             12.11                       12.90   
3                             12.07                        9.60   
4                             11.15                        7.36   
5                              9.15                        5.44   

   AVERAGE Air Temperature DegC  
1                         16.01  
2                         16.08  
3                         16.05  
4                         15.25  
5                         13.34

In [62]:
# This is the final data which the model will be built on
data.to_csv('sample_test_heat_stress_v2.csv')